<a href="https://colab.research.google.com/github/dangtrantrung/Tuning---HyperParameter-Optimization-with-Grid-Search-Keras-Tuner/blob/master/Tuning_HyperParameter_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WorkFlow
## 1. Load Data
## 2. Build Hyper Model with Hyper Parameters
## 3. Select Tuner/Grid Search
## 4. Train Model with Data
## 5. Build model with Best HP
## 6. Tuning to get best HP

In [32]:

!pip install keras_tuner

In [61]:

import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

In [56]:
# import data

(img_train,label_train),(img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize
img_train=img_train.astype('float32')/255
img_test=img_test.astype('float32')/255

In [57]:
label_train[8]

5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.5 MB/s eta 0:00:00


In [65]:
# build Model HP
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  X = hp.Int("units", min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=X, activation="relu"))
  model.add(keras.layers.Dense(10))

  lr = hp.Choice("lr",values=[1e-2,1e-3,1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss = 'spare_categorical_crossentropy', metrics=["accuracy"])
  return model



In [74]:
tuner = kt.Hyperband (model_builder, objective="val_accuracy", directory="tuner_dir_01", project_name="Demo_Tuner_01")

# Keras Callbacks - Early Stop
stop_early = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)



ModuleNotFoundError: No module named 'keras.src.engine.base_layer_v1'

In [72]:
!rm rf /content/tuner_dir_01

rm: cannot remove 'rf': No such file or directory
rm: cannot remove '/content/tuner_dir_01': No such file or directory


In [75]:
tuner.search( img_train, label_train, validation_split=0.2, callbacks=[stop_early])


NameError: name 'tuner' is not defined